### Installing LangChain

In [1]:
!pip3 install -q langchain==0.0.152

### LangChain library uses PyLLaMAcpp module to load the converted GPT4All weights

In [2]:
!pip3 install -q pyllamacpp==1.0.7

### Installing sentencepiece

In [3]:
!pip install -q sentencepiece

### Importing required modules

In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Setting up the tempelate

In [5]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

### Providing the local path

In [6]:
local_path = './models/gpt4all-lora-quantized-ggml.bin'  

### Importing requests to fetch the url

In [7]:
import requests
from pathlib import Path
from tqdm import tqdm

local_path = './models/gpt4all-lora-quantized-ggml.bin'
Path(local_path).parent.mkdir(parents=True, exist_ok=True)

url = 'https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized-ggml.bin'

# send a GET request to the URL to download the file. Stream since it's large
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response to it in chunks
# This is a large file, so have to wait.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)

514266it [01:15, 6825.62it/s]


https://github.com/ggerganov/llama.cpp#using-gpt4all

### Clone llama.cpp Repository and Checkout Commit 2b26469.

In [8]:
!git clone https://github.com/ggerganov/llama.cpp.git
!cd llama.cpp && git checkout 2b26469

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
HEAD is now at 2b264693 convert.py: Support models which are stored in a single pytorch_model.bin (#1469)


### Convert Model with llama.cpp Script.

In [9]:
!python3 llama.cpp/convert.py ./models/gpt4all-lora-quantized-ggml.bin

Loading model file models/gpt4all-lora-quantized-ggml.bin
Writing vocab...
[  1/291] Writing tensor tok_embeddings.weight                  | size  32001 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  2/291] Writing tensor norm.weight                            | size   4096           | type UnquantizedDataType(name='F32')
[  3/291] Writing tensor output.weight                          | size  32001 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  4/291] Writing tensor layers.0.attention.wq.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  5/291] Writing tensor layers.0.attention.wk.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  6/291] Writing tensor layers.0.attention.wv.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addend

### Initialize Callback Manager for Streaming Output.

In [10]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

### Initialize GPT4All Model with Callbacks and Verbose Output.

In [11]:
# Verbose is required to pass to the callback manager
# When verbose output is enabled, the model will give us more information as it runs.
llm = GPT4All(model="./models/ggml-model-q4_0.bin", callback_manager=callback_manager, verbose=True)

### Creating an LLMChain using a given prompt and the previously initialized GPT4All model.

In [12]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

## Examples

### Mother's day Example

In [17]:
question = "Write a social media post to celebrate mother's day."

In [18]:
llm_chain.run(question)

 Question: Write a social media post to celebrate mother's day.

Answer: Let's think step by step. First, we must address the audience; let’s start with a hashtag that connects us all—#HappyMothersDay or #InternationalWomensMonth can be used for this purpose as well. Nowadays social media is full of posts and tweets to celebrate Mother's Day, so we must make our post noticeable; therefore it could be good idea including a picture that represents motherhood such as kids playing with their moms or grandmothers taking care about them like the above image. We can also include an uplifting quote from famous personality who is known for his/her kindness towards mothers, which we feel would serve our purpose of wishing a Happy Mother's Day to everyone across social media platforms; this could be a good way too!

" Question: Write a social media post to celebrate mother's day.\n\nAnswer: Let's think step by step. First, we must address the audience; let’s start with a hashtag that connects us all—#HappyMothersDay or #InternationalWomensMonth can be used for this purpose as well. Nowadays social media is full of posts and tweets to celebrate Mother's Day, so we must make our post noticeable; therefore it could be good idea including a picture that represents motherhood such as kids playing with their moms or grandmothers taking care about them like the above image. We can also include an uplifting quote from famous personality who is known for his/her kindness towards mothers, which we feel would serve our purpose of wishing a Happy Mother's Day to everyone across social media platforms; this could be a good way too!"

### Explain Rain Example

In [19]:
question = "What happens when it rains somewhere?"

In [20]:
llm_chain.run(question)

 Question: What happens when it rains somewhere?

Answer: Let's think step by step. Whenever there is rain, clouds and moisture in the air create a phenomenon called precipitation (rain or snow). This type of weather usually occurs due to changes within an area where temperature differences are too great for natural convection processes alone to handle. The resultant water either falls as raindrops from clouds directly above it, or as snowflakes if the air is cold enough and moisture high in the atmosphere condenses into ice crystals which float downwards (snow). Precipitation then flows overland through a river network to lakes, seas or oceans. The amount of precipitation that occurs depends on factors such as temperature differences between air masses, humidity levels and wind patterns - among others!

" Question: What happens when it rains somewhere?\n\nAnswer: Let's think step by step. Whenever there is rain, clouds and moisture in the air create a phenomenon called precipitation (rain or snow). This type of weather usually occurs due to changes within an area where temperature differences are too great for natural convection processes alone to handle. The resultant water either falls as raindrops from clouds directly above it, or as snowflakes if the air is cold enough and moisture high in the atmosphere condenses into ice crystals which float downwards (snow). Precipitation then flows overland through a river network to lakes, seas or oceans. The amount of precipitation that occurs depends on factors such as temperature differences between air masses, humidity levels and wind patterns - among others!"

### Rain Example, but in two sentences in funny way.

In [21]:
#Telling the tempelate to reply in fun way. 
template = """Question: {question}

Answer: Let's answer in two sentence while being funny."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [22]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [23]:
llm_chain.run(question)

 Question: What happens when it rains somewhere?

Answer: Let's answer in two sentence while being funny. 1) It starts to rain, and the clouds become black because of all that water falling from them onto Earth’s surface or something like that (I don’t know how it works). Or...2) The weather becomes uncomfortable for everyone including those who are in their offices, on the streets, driving cars etc.

" Question: What happens when it rains somewhere?\n\nAnswer: Let's answer in two sentence while being funny. 1) It starts to rain, and the clouds become black because of all that water falling from them onto Earth’s surface or something like that (I don’t know how it works). Or...2) The weather becomes uncomfortable for everyone including those who are in their offices, on the streets, driving cars etc."

# Thank You